In [7]:
from sqlalchemy import create_engine, Column, String, Float, Integer
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [8]:
import uuid
import time
import os 
from pathlib import Path


In [9]:
Base = declarative_base()

class VehicleState(Base):
    """车辆状态数据表"""
    __tablename__ = 'vehicle_state'

    token = Column(String(36), primary_key=True)
    timestamp = Column(Float, nullable=False)
    x = Column(Float, nullable=False)
    y = Column(Float, nullable=False)
    heading = Column(Float, nullable=False)
    velocity = Column(Float, nullable=False)
    frame_id = Column(Integer, nullable=False)

    def __repr__(self):
        return f"VehicleState(frame={self.frame_id}, pos=({self.x:.2f}, {self.y:.2f}))"


class LidarData(Base):
    """激光雷达数据表"""
    __tablename__ = 'lidar'

    token = Column(String(36), primary_key=True)
    timestamp = Column(Float, nullable=False)
    filename = Column(String(255), nullable=False)
    num_points = Column(Integer, nullable=False)
    frame_id = Column(Integer, nullable=False)

    def __repr__(self):
        return f"LidarData(frame={self.frame_id}, points={self.num_points})"


In [10]:
# define the db file path
# generate an db file which contains the predefined data class above 
db_dir = Path("./")
db_file = db_dir / "autodrive_scene.db"
engine = create_engine(f'sqlite:///{db_file.absolute()}')
Base.metadata.create_all(engine)


In [11]:
# 创建会话，并通过sessionmake对数据进行操作
Session = sessionmaker(bind=engine)
session = Session()

# 生成一帧数据
timestamp = time.time()
frame_token = str(uuid.uuid4())

print("timestamp: ", timestamp)
print("frame_token: ", frame_token)
# 车辆状态数据
vehicle_state = VehicleState(
    token=frame_token,
    frame_id=int(1),
    timestamp=timestamp,
    x=100.5,
    y=200.3,
    heading=1.57,  # 约90度
    velocity=10.0,
)

# Lidar数据
lidar_data = LidarData(token=frame_token,
                      frame_id = int(1),
                       timestamp=timestamp,
                       filename=f'lidar/{frame_token}.pcd',
                       num_points=100000
                        )


timestamp:  1739114190.315595
frame_token:  d470cada-e4d5-4bb0-a206-72c2055928bc


In [12]:
# 将数据添加到数据库
session.add(vehicle_state)
session.add(lidar_data)
session.commit()
session.close()